# 🎭 Multimodal AI Synthetic Data Generator

A powerful tool for generating realistic synthetic datasets using multiple AI models via OpenRouter.

**Features:**
- 📊 Multiple dataset types (Customer Records, Financial Transactions, etc.)
- 🤖 Multiple AI models (OpenAI, Gemini, Grok)
- 🔄 Robust JSON extraction and error handling
- 📥 Download generated data as CSV
- 🎨 Clean Gradio web interface

## 📦 Install Dependencies

In [ ]:
!pip install openai pandas gradio python-dotenv

## 📚 Import Libraries

In [16]:
import os
import json
import pandas as pd
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI

# Import our custom library
from data_generator_lib import DataGenerator, generate_data, extract_json_from_text, clean_response_text

## 🔐 Environment Setup

In [17]:
# Load environment variables
load_dotenv()

# Check for API key
api_key = os.getenv("OPENROUTER_API_KEY")
if not api_key:
    raise EnvironmentError("OPENROUTER_API_KEY not found in .env file")

print("✅ Environment loaded successfully")

✅ Environment loaded successfully


## 🤖 Initialize OpenRouter Client

In [18]:
# Initialize OpenRouter client
client = OpenAI(
    api_key=api_key,
    base_url="https://openrouter.ai/api/v1"
)

print("✅ OpenRouter client initialized")

✅ OpenRouter client initialized


## 🎯 Available Models

Define models from different providers for synthetic data generation.

In [19]:
# Model configurations
MODELS = {
    "OpenAI (GPT-4o Mini)": "openai/gpt-4o-mini",
    "Gemini Flash": "google/gemini-2.5-flash",
    "Grok Beta": "x-ai/grok-3-mini",
    "Claude Haiku": "anthropic/claude-3-haiku-20240307",
    "DeepSeek V3": "deepseek/deepseek-r1"
}

print(f"✅ Loaded {len(MODELS)} models")

✅ Loaded 5 models


## 📝 Professional Prompt Templates

Creates structured prompts for accurate, usable datasets with strict JSON formatting instructions.

In [20]:
def build_prompt(dataset_type, size):
    """
    Build a structured prompt for the AI model.
    
    Args:
        dataset_type: Type of dataset to generate
        size: Number of records to generate
        
    Returns:
        Formatted prompt string
    """
    
    # Define the example as a separate string to avoid f-string issues
    customer_example = '''[
  {
    "customer_id": "123e4567-e89b-12d3-a456-426614174000",
    "full_name": "John Smith",
    "email": "john.smith@email.com",
    "phone": "+1-555-123-4567",
    "country": "USA",
    "city": "New York",
    "postal_code": "10001",
    "signup_date": "2023-05-15"
  }
]'''
    
    transaction_example = '''[
  {
    "transaction_id": "TXN-8f3a2b1c",
    "account_id": "ACC-987654",
    "transaction_date": "2024-02-15",
    "amount": 47.85,
    "currency": "USD",
    "merchant_name": "Walmart",
    "merchant_category": "Groceries",
    "transaction_type": "debit",
    "status": "completed"
  }
]'''
    
    tax_example = '''[
  {
    "taxpayer_id": "TAX-123456",
    "full_name": "Jane Doe",
    "country": "USA",
    "annual_income": 75000,
    "tax_paid": 15750,
    "employment_status": "employed",
    "filing_status": "single",
    "tax_year": 2023
  }
]'''
    
    healthcare_example = '''[
  {
    "patient_id": "PAT-789012",
    "full_name": "Robert Johnson",
    "age": 58,
    "gender": "Male",
    "blood_type": "O+",
    "diagnosis": "Hypertension",
    "admission_date": "2023-11-10",
    "country": "Canada"
  }
]'''
    
    ecommerce_example = '''[
  {
    "order_id": "ORD-543210",
    "customer_id": "CUST-98765",
    "order_date": "2024-01-20",
    "product_name": "Wireless Headphones",
    "category": "Electronics",
    "quantity": 1,
    "price": 89.99,
    "payment_method": "credit_card",
    "shipping_country": "USA"
  }
]'''
    
    templates = {
        "Customer Records": f"""
Generate {size} realistic customer records as a JSON array.

**CRITICAL INSTRUCTION:** Return ONLY a valid JSON array. No explanations, no markdown formatting, no backticks, no additional text.

**REQUIRED FIELDS (each object must include):**
- customer_id: UUID format (e.g., "123e4567-e89b-12d3-a456-426614174000")
- full_name: string (first and last name)
- email: string (valid email format)
- phone: string (include country code, e.g., "+1-555-123-4567")
- country: string
- city: string
- postal_code: string
- signup_date: string (YYYY-MM-DD format, between 2020-01-01 and today)

**DATA RULES:**
- Use double quotes (\") for strings, not single quotes
- Include diverse, realistic values across different countries and cities
- Ensure all emails are properly formatted

**EXAMPLE OUTPUT:**
{customer_example}

Return ONLY the JSON array with {size} objects:
""",

        "Financial Transactions": f"""
Generate {size} realistic banking transactions as a JSON array.

**CRITICAL INSTRUCTION:** Return ONLY a valid JSON array. No explanations, no markdown formatting, no backticks, no additional text.

**REQUIRED FIELDS:**
- transaction_id: string (format: TXN- followed by random chars)
- account_id: string (format: ACC- followed by digits)
- transaction_date: string (YYYY-MM-DD, within last 30 days)
- amount: number (decimal with 2 places, $5-$500 for purchases, up to $5000 for transfers)
- currency: string (mostly "USD", occasional "EUR", "GBP")
- merchant_name: string
- merchant_category: string (e.g., "Groceries", "Restaurant", "Retail", "Utilities")
- transaction_type: string ("debit" or "credit", ~60% debit, 40% credit)
- status: string ("completed", "pending", or "failed", 95% completed)

**EXAMPLE OUTPUT:**
{transaction_example}

Return ONLY the JSON array:
""",

        "Tax Payer Records": f"""
Generate {size} realistic taxpayer records as a JSON array.

**CRITICAL INSTRUCTION:** Return ONLY a valid JSON array. No explanations, no markdown formatting, no backticks, no additional text.

**REQUIRED FIELDS:**
- taxpayer_id: string (format: TAX- followed by digits)
- full_name: string
- country: string
- annual_income: integer (should correlate with employment status)
- tax_paid: integer (roughly 15-30% of income)
- employment_status: string ("employed", "self-employed", "retired", "unemployed")
- filing_status: string ("single", "married_filing_jointly", "head_of_household")
- tax_year: integer (2023 or 2022)

**EXAMPLE OUTPUT:**
{tax_example}

Return ONLY the JSON array:
""",

        "Healthcare Patients": f"""
Generate {size} realistic patient records as a JSON array.

**CRITICAL INSTRUCTION:** Return ONLY a valid JSON array. No explanations, no markdown formatting, no backticks, no additional text.

**REQUIRED FIELDS:**
- patient_id: string (format: PAT- followed by digits)
- full_name: string
- age: integer (0-90, realistic distribution)
- gender: string ("Male", "Female", or "Other")
- blood_type: string (A+, A-, B+, B-, AB+, AB-, O+, O-)
- diagnosis: string (common conditions: hypertension, diabetes, asthma, etc.)
- admission_date: string (YYYY-MM-DD, within last 5 years)
- country: string

**EXAMPLE OUTPUT:**
{healthcare_example}

Return ONLY the JSON array:
""",

        "E-commerce Orders": f"""
Generate {size} realistic e-commerce orders as a JSON array.

**CRITICAL INSTRUCTION:** Return ONLY a valid JSON array. No explanations, no markdown formatting, no backticks, no additional text.

**REQUIRED FIELDS:**
- order_id: string (format: ORD- followed by digits)
- customer_id: string (format: CUST- followed by digits)
- order_date: string (YYYY-MM-DD, within last 90 days)
- product_name: string
- category: string (Electronics, Clothing, Books, Home & Garden, Sports, Beauty)
- quantity: integer (1-5)
- price: number (decimal with 2 places, realistic for category)
- payment_method: string ("credit_card", "paypal", "apple_pay", "google_pay")
- shipping_country: string

**EXAMPLE OUTPUT:**
{ecommerce_example}

Return ONLY the JSON array:
"""
    }

    if dataset_type not in templates:
        raise ValueError(f"Unknown dataset type: {dataset_type}")
        
    return templates[dataset_type]

## 🚀 Initialize Data Generator

Create an instance of our DataGenerator class with robust error handling.

In [21]:
# Create generator instance
generator = DataGenerator(client, MODELS, build_prompt)

print("✅ Data generator initialized")
print(f"   Available models: {', '.join(MODELS.keys())}")
print(f"   Dataset types: Customer Records, Financial Transactions, Tax Payer Records, Healthcare Patients, E-commerce Orders")

✅ Data generator initialized
   Available models: OpenAI (GPT-4o Mini), Gemini Flash, Grok Beta, Claude Haiku, DeepSeek V3
   Dataset types: Customer Records, Financial Transactions, Tax Payer Records, Healthcare Patients, E-commerce Orders


## 🧪 Test Generation

Verify that data generation works correctly.

In [22]:
# Test with a small sample
print("Testing data generation...")
test_df, preview = generator.generate_with_preview(
    model_name="OpenAI (GPT-4o Mini)",
    dataset_type="Customer Records",
    size=3,
    verbose=True
)

if 'error' in test_df.columns and not test_df['error'].isna().all():
    print("\n❌ Test failed:")
    print(test_df[['error', 'error_type']].to_string(index=False))
else:
    print("\n✅ Test successful!")
    print(f"Generated {len(test_df)} records with columns: {list(test_df.columns)}")
    display(test_df.head())

Testing data generation...
Attempt 1/3 for Customer Records (3 records)
Raw response preview: [
  {
    "customer_id": "f47ac10b-58cc-4372-a567-0e02b2c3d479",
    "full_name": "Alice Johnson",
    "email": "alice.johnson@example.com",
    "phone": "+44-20-7946-0958",
    "country": "United Kin...

✅ Test successful!
Generated 3 records with columns: ['customer_id', 'full_name', 'email', 'phone', 'country', 'city', 'postal_code', 'signup_date']


,customer_id,full_name,email,phone,country,city,postal_code,signup_date
0,f47ac10b-58cc-4372-a567-0e02b2c3d479,Alice Johnson,alice.johnson@example.com,+44-20-7946-0958,United Kingdom,London,EC1A 1BB,2021-03-22
1,b2f1c5c5-2f3e-4b3e-a7c1-3f7f5e1e8f9a,Carlos Martinez,carlos.martinez@email.com,+34-91-123-4567,Spain,Madrid,28001,2022-07-10
2,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,Yuki Tanaka,yuki.tanaka@example.co.jp,+81-3-1234-5678,Japan,Tokyo,100-0001,2020-11-05


## 🎨 Build Gradio Interface

Create an interactive web UI for the data generator.

In [23]:
def generate_dataset_ui(model_name, dataset_type, size):
    """
    UI function for Gradio interface.
    
    Args:
        model_name: Selected model name
        dataset_type: Selected dataset type
        size: Number of records
        
    Returns:
        Tuple of (DataFrame preview, CSV file path)
    """
    # Generate data
    df = generator.generate(
        model_name=model_name,
        dataset_type=dataset_type,
        size=size,
        max_retries=3,
        temperature=0.4
    )
    
    # Save to CSV
    filename = f"synthetic_{dataset_type.lower().replace(' ', '_')}_{size}.csv"
    df.to_csv(filename, index=False)
    
    return df, filename


# Define dataset options
DATASET_TYPES = [
    "Customer Records",
    "Financial Transactions",
    "Tax Payer Records",
    "Healthcare Patients",
    "E-commerce Orders"
]

# Create Gradio interface
with gr.Blocks(title="AI Synthetic Data Generator", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🎭 Multimodal AI Synthetic Data Generator")
    gr.Markdown("Generate realistic datasets using multiple AI models via OpenRouter.")
    
    with gr.Row():
        with gr.Column():
            model_dropdown = gr.Dropdown(
                choices=list(MODELS.keys()),
                value=list(MODELS.keys())[0],
                label="🤖 Select AI Model",
                info="Choose the AI model for generation"
            )
            
            dataset_dropdown = gr.Dropdown(
                choices=DATASET_TYPES,
                value=DATASET_TYPES[0],
                label="📊 Dataset Type",
                info="Select the type of synthetic data to generate"
            )
            
            size_slider = gr.Slider(
                minimum=5,
                maximum=50,
                value=50,
                step=5,
                label="📏 Number of Records",
                info="How many records to generate (5-50)"
            )
            
            generate_btn = gr.Button("🚀 Generate Dataset", variant="primary")
        
        with gr.Column():
            preview_output = gr.Dataframe(
                label="👁️ Data Preview",
                interactive=False,
                wrap=True
            )
            
            download_output = gr.File(
                label="📥 Download CSV",
                interactive=False
            )
    
    gr.Markdown("---")
    gr.Markdown("### 📋 Instructions")
    gr.Markdown("""
    1. Select an AI model from the dropdown
    2. Choose the type of dataset you want to generate
    3. Set the number of records (5-500)
    4. Click 'Generate Dataset' and wait a few seconds
    5. Preview the data and download as CSV
    """)
    
    # Connect the button to the function
    generate_btn.click(
        fn=generate_dataset_ui,
        inputs=[model_dropdown, dataset_dropdown, size_slider],
        outputs=[preview_output, download_output]
    )
    
    gr.Markdown("---")
    gr.Markdown("Built with OpenRouter, Gradio, and ❤️ from Yaqub")

C:\Users\user\AppData\Local\Temp\ipykernel_21188\365295536.py:39: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(title="AI Synthetic Data Generator", theme=gr.themes.Soft()) as demo:


## ▶️ Launch the App

Start the Gradio web interface.

In [ ]:
# Launch the app
if __name__ == "__main__":
    demo.launch(
        share=False,  # Set to True for public link
        server_name="127.0.0.1",
        server_port=7860,
        debug=True
    )

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## 🔧 Troubleshooting

If you encounter JSON parsing errors, run this diagnostic cell to see what the model is returning:

In [ ]:
def debug_model_response(model_name, dataset_type, size=2):
    """
    Debug function to see raw model response.
    """
    prompt = build_prompt(dataset_type, size)
    
    print(f"=== DEBUG: {model_name} generating {dataset_type} ===\n")
    
    try:
        response = client.chat.completions.create(
            model=MODELS[model_name],
            messages=[{"role": "user", "content": prompt}],
            temperature=0.4,
        )
        
        raw_text = response.choices[0].message.content
        print("RAW RESPONSE (first 500 chars):")
        print("-" * 50)
        print(raw_text[:500])
        print("-" * 50)
        
        # Try our extraction
        cleaned = clean_response_text(raw_text)
        extracted = extract_json_from_text(raw_text)
        
        print(f"\nCleaned text length: {len(cleaned)}")
        print(f"Extraction successful: {extracted is not None}")
        
        if extracted:
            print(f"Extracted type: {type(extracted)}")
            if isinstance(extracted, list):
                print(f"Number of items: {len(extracted)}")
        
    except Exception as e:
        print(f"Error: {type(e).__name__}: {e}")

# Run debug
debug_model_response("OpenAI (GPT-4o Mini)", "Customer Records", 2)